In [33]:
with open('text.txt', 'r', encoding="utf-8") as f:
    text = f.read()

In [34]:
print('lenth of dataset in characters: ', len(text))

lenth of dataset in characters:  11853750


In [35]:
#Printing first 100 characters
print(text[:100])

Albert of Prussia ( 17 May 1490 20 March 1568 ) was the last Grand Master of the Teutonic Knights , 


In [36]:
#All unique chr in text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 !#$%&'()*+,-./0123456789:;<=>?@ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyz{
86


In [37]:
#mapping of char to int
stoi = { ch:i for i, ch in enumerate(chars)}
itos = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s] # str to int list
decode = lambda l: ''.join([itos[i] for i in l]) #int to str

print(encode("Hello"))
print(decode(encode("Hello")))

[40, 63, 70, 70, 73]
Hello


In [38]:
#Encoding entire text in dataset and storing in a torch.tensor
import torch
data = torch.tensor(encode(text), dtype=torch.long)
print (data.shape,data.type)
print(data[:100])

torch.Size([11853750]) <built-in method type of Tensor object at 0x0000020A9D52F920>
tensor([33, 70, 60, 63, 76, 78,  1, 73, 64,  1, 48, 76, 79, 77, 77, 67, 59,  1,
         8,  1, 17, 23,  1, 45, 59, 83,  1, 17, 20, 25, 16,  1, 18, 16,  1, 45,
        59, 76, 61, 66,  1, 17, 21, 22, 24,  1,  9,  1, 81, 59, 77,  1, 78, 66,
        63,  1, 70, 59, 77, 78,  1, 39, 76, 59, 72, 62,  1, 45, 59, 77, 78, 63,
        76,  1, 73, 64,  1, 78, 66, 63,  1, 52, 63, 79, 78, 73, 72, 67, 61,  1,
        43, 72, 67, 65, 66, 78, 77,  1, 12,  1])


In [39]:
#split of data
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [40]:
block_size = 8
train_data[:block_size+1]

tensor([33, 70, 60, 63, 76, 78,  1, 73, 64])

In [41]:
x = train_data[:block_size]
y = train_data[1:block_size+1]
for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([33]) the target: 70
when input is tensor([33, 70]) the target: 60
when input is tensor([33, 70, 60]) the target: 63
when input is tensor([33, 70, 60, 63]) the target: 76
when input is tensor([33, 70, 60, 63, 76]) the target: 78
when input is tensor([33, 70, 60, 63, 76, 78]) the target: 1
when input is tensor([33, 70, 60, 63, 76, 78,  1]) the target: 73
when input is tensor([33, 70, 60, 63, 76, 78,  1, 73]) the target: 64


In [42]:
torch.manual_seed(1337)
batch_size = 4 #independent seq process in parallel
block_size = 8 #max length for prediction

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    return x, y

xb, yb = get_batch("train")
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)

print('----')

for b in range(batch_size):         #batch dimension
    for t in range(block_size):     #time dimension
        context = xb[b, :t+1]
        target = yb[b,t]
        print(f'when input is {context.tolist()} the target: {target}')

inputs:
torch.Size([4, 8])
tensor([[14,  1, 40, 63,  1, 59, 78, 78],
        [80, 59,  1, 14,  1, 52, 66, 63],
        [81, 59, 77,  1, 72, 73, 78,  1],
        [67, 73, 72, 59, 70, 63,  1,  8]])
targets:
torch.Size([4, 8])
tensor([[ 1, 40, 63,  1, 59, 78, 78, 63],
        [59,  1, 14,  1, 52, 66, 63,  1],
        [59, 77,  1, 72, 73, 78,  1, 73],
        [73, 72, 59, 70, 63,  1,  8,  1]])
----
when input is [14] the target: 1
when input is [14, 1] the target: 40
when input is [14, 1, 40] the target: 63
when input is [14, 1, 40, 63] the target: 1
when input is [14, 1, 40, 63, 1] the target: 59
when input is [14, 1, 40, 63, 1, 59] the target: 78
when input is [14, 1, 40, 63, 1, 59, 78] the target: 78
when input is [14, 1, 40, 63, 1, 59, 78, 78] the target: 63
when input is [80] the target: 59
when input is [80, 59] the target: 1
when input is [80, 59, 1] the target: 14
when input is [80, 59, 1, 14] the target: 1
when input is [80, 59, 1, 14, 1] the target: 52
when input is [80, 59, 1, 1

In [43]:
import torch 
import torch.nn as nn
from torch.nn import functional as F
torch.manual_seed(1337)

class BigramLanguageModel(nn.Module):

    def __init__(self,vocab_size):
        super().__init__()
        #each token directly reads off the logits for next token from lookup table
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):

        # idx and targets are both (B,T) tensor of integers
        logits = self.token_embedding_table(idx) # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss
    
    def generate(self, idx, max_new_tokens):
            # idx is (B, T) array of indices in the current context
            for _ in range(max_new_tokens):
                # get the predictions
                logits, loss = self(idx)
                # focus only on the last time step
                logits = logits[:, -1, :] # becomes (B, C)
                # apply softmax to get probabilities
                probs = F.softmax(logits, dim=-1) # (B, C)
                # sample from the distribution
                idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
                # append sampled index to the running sequence
                idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
            return idx

m = BigramLanguageModel(vocab_size)
logits, loss = m(xb,yb)
print(logits.shape)
print(loss)

print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=100)[0].tolist()))

torch.Size([32, 86])
tensor(4.6551, grad_fn=<NllLossBackward0>)

?975,0uO=+1?mfqhT-K*:>Q/a?*xR%MxTDe&Sj/alo+>:pZkaN#$Db7a%l5*:budALArM230yt.b*>xWh;6&W#dp=k8,b:QFe9.*


In [44]:
# create optimizer
optimizer = torch.optim.Adam(m.parameters(), lr=1e-3)

In [45]:
batch_size = 32
for steps in range(10000): # increase number of steps for good results...

    # sample a batch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

print(loss.item())

2.4811253547668457


In [46]:
print(decode(m.generate(idx = torch.zeros((1, 1), dtype=torch.long), max_new_tokens=400)[0].tolist()))



berioreathastwichamods 2&kind1066, we . abediony @ ) AGeary onis fonds FFo Himofre angemer Cme atin Ouen tcklla$100 qul l prive @ d Naluringatonime ' .Shouand telil HItista G FArs lineelsondNommaly @ 10 f an-bs wasedim , ** Bache as ara re ondit ioraue Cofepond astaneathan ' : @ we . Pr oled bey aunthind oum # wilalocaly @ @ himanece f Can @ intou' 2099066 . , S. w Fof cangunc Al aralisth . omervi


Math Trick

In [63]:
# toy example illustrating how matrix multiplication can be used for a "weighted aggregation"
torch.manual_seed(42)
a = torch.tril(torch.ones(3, 3))
a = a / torch.sum(a, 1, keepdim=True)
b = torch.randint(0,10,(3,2)).float()
c = a @ b
print('a=')
print(a)
print('--')
print('b=')
print(b)
print('--')
print('c=')
print(c)

a=
tensor([[1.0000, 0.0000, 0.0000],
        [0.5000, 0.5000, 0.0000],
        [0.3333, 0.3333, 0.3333]])
--
b=
tensor([[2., 7.],
        [6., 4.],
        [6., 5.]])
--
c=
tensor([[2.0000, 7.0000],
        [4.0000, 5.5000],
        [4.6667, 5.3333]])


In [64]:
# consider the following toy example:

torch.manual_seed(1337)
B,T,C = 4,8,2 # batch, time, channels
x = torch.randn(B,T,C)
x.shape

torch.Size([4, 8, 2])

In [65]:
# We want x[b,t] = mean_{i<=t} x[b,i]
xbow = torch.zeros((B,T,C))
for b in range(B):
    for t in range(T):
        xprev = x[b,:t+1] # (t,C)
        xbow[b,t] = torch.mean(xprev, 0)


In [73]:
# version 2: using matrix multiply for a weighted aggregation
wei = torch.tril(torch.ones(T, T))
wei = wei / wei.sum(1, keepdim=True)
xbow2 = wei @ x # (B, T, T) @ (B, T, C) ----> (B, T, C)
torch.allclose(xbow, xbow2, atol=1e-6)

True

In [67]:
# version 3: use Softmax
tril = torch.tril(torch.ones(T, T))
wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)
xbow3 = wei @ x
torch.allclose(xbow2, xbow3)

True

In [74]:
# version 4: self-attention!
torch.manual_seed(1337)
B,T,C = 4,8,32 # batch, time, channels
x = torch.randn(B,T,C)

# let's see a single Head perform self-attention
head_size = 16
key = nn.Linear(C, head_size, bias=False)
query = nn.Linear(C, head_size, bias=False)
value = nn.Linear(C, head_size, bias=False)
k = key(x)   # (B, T, 16)
q = query(x) # (B, T, 16)
wei =  q @ k.transpose(-2, -1) # (B, T, 16) @ (B, 16, T) ---> (B, T, T)

tril = torch.tril(torch.ones(T, T))
#wei = torch.zeros((T,T))
wei = wei.masked_fill(tril == 0, float('-inf'))
wei = F.softmax(wei, dim=-1)

v = value(x)
out = wei @ v
#out = wei @ x

out.shape

torch.Size([4, 8, 16])

In [75]:
wei[0]

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.1574, 0.8426, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2088, 0.1646, 0.6266, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5792, 0.1187, 0.1889, 0.1131, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0294, 0.1052, 0.0469, 0.0276, 0.7909, 0.0000, 0.0000, 0.0000],
        [0.0176, 0.2689, 0.0215, 0.0089, 0.6812, 0.0019, 0.0000, 0.0000],
        [0.1691, 0.4066, 0.0438, 0.0416, 0.1048, 0.2012, 0.0329, 0.0000],
        [0.0210, 0.0843, 0.0555, 0.2297, 0.0573, 0.0709, 0.2423, 0.2391]],
       grad_fn=<SelectBackward0>)